In [8]:
from cred import *
import sqlalchemy
from sqlalchemy.orm import sessionmaker
import spotipy
from twilio.rest import Client
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from datetime import *
import sqlite3

## Constant

In [2]:
SCOPE = "user-read-recently-played"
REDIRECT_URL = "http://localhost:7777/callback"
DATABASE_LOCATION = "sqlite:///my_played_tracks.sqlite"
client = Client(ACCOUNT_SID, AUTH_TOKEN)

# Helper Function

In [3]:
def check_if_valid_data(df: pd.DataFrame) -> bool:
    """
    Validate the data
    """

    # is data empty?
    if df.empty:
        print("No songs downloaded. Finishing execution.")
        return False
     # primary key constraint
    if pd.Series(df['played_at']).is_unique:
        pass
    else:
        raise Exception("Primary key constraint is violated!")

    # check for nulls
    if df.isnull().values.any():
        raise Exception("Missing values exist!")
    
    # Check that all timestamps are of past five days
    today = date.today()
    last_five_days_list = [str(today - timedelta(days = day)) for day in range(5)]
    

    timestamps_list = df["timestamp"].tolist()
    if not any(item in timestamps_list for item in last_five_days_list):
            raise Exception("At least one of the returned songs does not have timestamp from last five days")
    return True

def send_msg(df) -> str:
    if not (df['artist_name'] == "Burna Boy").any():
        message = client.messages.create(
                              body='WOW! You have not listen to Burna Boy in the last five days',
                             from_= {FROM_NUMBER},     
                              to={TO_NUMBER} 
                          ) 
        return str(message)

In [4]:
if __name__ == "__main__":

    # set up connection
    sp = (spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
     client_secret= CLIENT_SECRET, redirect_uri=REDIRECT_URL, scope=SCOPE)))

    # I want to see what I have played in the last 5 days
    today = datetime.now()
    last_five_days = today - timedelta(days=5)
    last_five_unix = int(last_five_days.timestamp()) * 1000

    # Download all songs you've listened to "after yesterday", which means in the last 24 hours      
    data = sp.current_user_recently_played(limit=50, after={last_five_unix})

    song_names = []
    artist_names = []
    played_at_list = []
    timestamps = []

    # Extracting only the relevant bits of data from the json object
    for idx, item in enumerate(data['items']):

        track = item['track']
        played_at = item['played_at']

        song_names.append(track['name'])
        artist_names.append(track['artists'][0]['name'])
        played_at_list.append(played_at)
        timestamps.append(played_at[0:10])
        
    

    # Prepare a dictionary in order to turn it into a pandas dataframe below       
    song_dict = {
        "song_name" : song_names,
        "artist_name": artist_names,
        "played_at" : played_at_list,
        "timestamp" : timestamps,
        "dt": pd.to_datetime("today").strftime("%Y-%m-%d")
    }    
     
    df = pd.DataFrame(song_dict, columns = [ "artist_name","song_name", "played_at", "timestamp", "dt"])


    # validate
    if check_if_valid_data(df):
        print("Data validate proceed to Load stage")

    # msg
    if send_msg(df):
        print("SMS sent!")
    else:
        print("SMS not sent")
 
    # Load
    engine = sqlalchemy.create_engine(DATABASE_LOCATION)
    conn = sqlite3.connect('my_played_tracks.sqlite')
    cursor = conn.cursor()

    sql_query = """
    CREATE TABLE IF NOT EXISTS my_played_track(
        song_name VARCHAR(200),
        artist_name VARCHAR(200),
        played_at VARCHAR(200),
        timestamp VARCHAR(200),
        CONSTRAINT primary_key_constraint PRIMARY KEY (played_at)
    )
    """

    # cursor.execute(sql_query)
    print("Opened database successfully")

    try:
        df.to_sql("my_tracks", engine, index=False, if_exists='append')
    except:
        print("Data already exists in the database")

    conn.close()
    print("Close database successfully")
    

Data validate proceed to Load stage
SMS sent!
Opened database successfully
